In [1]:
import pandas as pd
#import dash
from jupyter_dash import JupyterDash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

launch_sites=spacex_df['Launch Site'].unique()#my definition

# Create a dash application
#app = dash.Dash(__name__)
app = JupyterDash(__name__)
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown', 
                                options=[{'label':x, 'value':x} for x in launch_sites], 
                                value='ALL', 
                                placeholder='Select a Launch Site here'),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',min=0, max=10000, step=1000,
                                               value=[min_payload,max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart'))
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'), 
    [Input(component_id='site-dropdown', component_property='value')])
def pie_Chart(value_pie):
            if value_pie=='ALL':   
                return px.pie(spacex_df, values=spacex_df.groupby(['Launch Site'])['class'].sum(), 
                          names=spacex_df['Launch Site'].unique(), 
                          title='All Launch Sites and Successes')
            else:
                return px.pie(spacex_df, values=spacex_df[spacex_df['Launch Site']==value_pie]['class'].value_counts(), 
                          names=spacex_df[spacex_df['Launch Site']==value_pie]['class'].unique(), 
                          title='Launch Site Successes and Failures')
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'), 
    [Input(component_id='site-dropdown', component_property='value'),
    Input(component_id='payload-slider', component_property='value')])
def scatter_graph (scatter_value, slider_value):
    if scatter_value=="ALL":
        return px.scatter(spacex_df, 
                          x=spacex_df[spacex_df['Payload Mass (kg)']<=slider_value[1]]['Payload Mass (kg)'] , 
                          y=spacex_df[spacex_df['Payload Mass (kg)']<=slider_value[1]]['class'], 
                          color=spacex_df[spacex_df['Payload Mass (kg)']<=slider_value[1]]['Booster Version Category'])
    else:
        return px.scatter(spacex_df, x=spacex_df[spacex_df['Launch Site']==scatter_value]['Payload Mass (kg)'], 
                          y=spacex_df[spacex_df['Launch Site']==scatter_value]['class'], 
                          color=spacex_df[spacex_df['Launch Site']==scatter_value]['Booster Version Category'])

# Run the app
#if __name__ == '__main__':
#    app.run_server()
app.run_server(mode='inline')